
# VIT-TINY_Training_VIA_ViT_SMALL_KD+CL

*Configuration*  
- Input resolution: *12-16-20-24-28-32*  
- Patch size: *2*

---

### Models & Parameters

- *Student (ViT‑Tiny)*  
  - Parameters: 5M  
  - Curriculum: 10 epochs per resolution stage  
  - 🎯 *Accuracy:* 79.92%

In [2]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy, RandomErasing
from transformers import (
    DeiTConfig,
    DeiTForImageClassification,
    ViTConfig,
    ViTPreTrainedModel,
    ViTModel
)
from tqdm import tqdm


use_dp = False  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

alpha, temp = 0.5, 4.0
stages = [(r, 10) for r in [12, 16, 20, 24, 28, 32]]


class ViTWithDistillation(ViTPreTrainedModel):
    def __init__(self, config: ViTConfig):
        super().__init__(config)
        self.vit = ViTModel(config)
        self.distill_token = nn.Parameter(torch.zeros(1, 1, config.hidden_size))
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.distiller = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(self, pixel_values, labels=None, teacher_logits=None,
                alpha=0.5, temperature=1.0):
        B = pixel_values.size(0)
        embeds = self.vit.embeddings(pixel_values, interpolate_pos_encoding=True)
        cls_emb, patch_emb = embeds[:, :1, :], embeds[:, 1:, :]
        dist_tok = self.distill_token.expand(B, -1, -1)
        x = torch.cat([cls_emb, dist_tok, patch_emb], dim=1)
        x = self.vit.encoder(x)[0]
        cls_out, dist_out = x[:, 0], x[:, 1]
        logits = self.classifier(cls_out)
        dist_logits = self.distiller(dist_out)
        output = {"logits": logits, "distill_logits": dist_logits}

        if labels is not None and teacher_logits is not None:
            loss_ce = F.cross_entropy(logits, labels)
            kd = F.kl_div(
                F.log_softmax(dist_logits / temperature, dim=1),
                F.softmax(teacher_logits / temperature, dim=1),
                reduction='batchmean'
            ) * (temperature ** 2)
            output["loss"] = (1 - alpha) * loss_ce + alpha * kd

        return output


teacher_config = DeiTConfig(
    image_size=32,
    patch_size=2,
    num_labels=10,
    hidden_size=384,
    num_hidden_layers=12,
    num_attention_heads=6,
    intermediate_size=1536,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    stochastic_depth_prob=0.1
)
teacher = DeiTForImageClassification(teacher_config).to(device)
if use_dp:
    teacher = nn.DataParallel(teacher)

ckpt = torch.load(
    "/kaggle/input/best-teacher/pytorch/default/1/best_teacher.pth",
    map_location=device
)
state = {k.replace('module.', ''): v for k, v in ckpt.items()}
teacher.load_state_dict(state, strict=True)
teacher.eval()
for p in teacher.parameters():
    p.requires_grad_(False)


tr_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    AutoAugment(policy=AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    RandomErasing(p=0.2, scale=(0.02,0.2), ratio=(0.3,3.3))
])
val_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

full_train = datasets.CIFAR10('./data', train=True, download=True, transform=tr_tf)
num_val = int(0.1 * len(full_train))
num_train = len(full_train) - num_val
train_ds, val_idx_ds = torch.utils.data.random_split(
    full_train, [num_train, num_val], generator=torch.Generator().manual_seed(42)
)
full_for_val = datasets.CIFAR10('./data', train=True, download=False, transform=val_tf)
val_ds = Subset(full_for_val, val_idx_ds.indices)

stu_cfg = ViTConfig(
    image_size=32,
    patch_size=2,
    num_labels=10,
    hidden_size=192,
    num_hidden_layers=12,
    num_attention_heads=3,
    intermediate_size=768,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    stochastic_depth_prob=0.1
)
student = ViTWithDistillation(config=stu_cfg).to(device)
opt = optim.AdamW(student.parameters(), lr=3e-4, weight_decay=1e-4)


for res, epochs in stages:
    print(f"\n--- Curriculum Stage: Crop {res}px for {epochs} epochs ---")
    train_stage_tf = transforms.Compose([
        transforms.RandomCrop(res, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    val_stage_tf = transforms.Compose([
        transforms.CenterCrop(res),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    base_train = datasets.CIFAR10('./data', train=True, download=False, transform=train_stage_tf)
    base_val = datasets.CIFAR10('./data', train=True, download=False, transform=val_stage_tf)
    tr_loader = DataLoader(Subset(base_train, train_ds.indices), batch_size=128, shuffle=True, num_workers=4)
    vl_loader = DataLoader(Subset(base_val, val_idx_ds.indices), batch_size=128, shuffle=False, num_workers=4)

    for ep in range(1, epochs + 1):
        # Training with progress bar
        student.train()
        for x, y in tqdm(tr_loader, desc=f"Stage {res}px Ep{ep}/{epochs}"):
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                tlog = teacher(pixel_values=x, interpolate_pos_encoding=True).logits
            out = student(pixel_values=x, labels=y, teacher_logits=tlog, alpha=alpha, temperature=temp)
            loss = out['loss'].mean()
            opt.zero_grad()
            loss.backward()
            opt.step()

        # Validation accuracy
        student.eval()
        correct, total = 0, 0
        for xb, yb in tqdm(vl_loader, desc=f"Val Stage {res}px Ep{ep}/{epochs}"):
            xb, yb = xb.to(device), yb.to(device)
            with torch.no_grad():
                preds = student(pixel_values=xb)['logits'].argmax(1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
        val_acc = 100 * correct / total
        print(f"Stage {res}px Ep{ep} val acc: {val_acc:.2f}%")


student.eval()
test_ds = datasets.CIFAR10('./data', train=False, transform=val_tf)
tloader = DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=4)
correct = 0
for xb, yb in tqdm(tloader, desc="Final Test"):
    xb, yb = xb.to(device), yb.to(device)
    with torch.no_grad():
        preds = student(pixel_values=xb)['logits'].argmax(1)
    correct += (preds == yb).sum().item()
student_acc = 100 * correct / len(test_ds)
print(f"Student Acc: {student_acc:.2f}%")


/tmp/ipykernel_31/283011062.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(


Files already downloaded and verified

--- Curriculum Stage: Crop 12px for 10 epochs ---


Val Stage 12px Ep1/10: 100%|██████████| 40/40 [00:00<00:00, 42.29it/s]


Stage 12px Ep1 val acc: 26.86%


Val Stage 12px Ep2/10: 100%|██████████| 40/40 [00:01<00:00, 38.48it/s]


...

Stage 32px Ep6 val acc: 79.14%


Val Stage 32px Ep7/10: 100%|██████████| 40/40 [00:04<00:00,  8.87it/s]


Stage 32px Ep7 val acc: 80.16%


Stage 32px Ep8/10:   1%|          | 3/352 [00:02<05:06,  1.14it/s]


KeyboardInterrupt: 

Early stopping done.

In [3]:
student.eval()
test_ds = datasets.CIFAR10('./data', train=False, transform=val_tf)
tloader = DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=4)
correct = 0
for xb, yb in tqdm(tloader, desc="Final Test"):
    xb, yb = xb.to(device), yb.to(device)
    with torch.no_grad():
        preds = student(pixel_values=xb)['logits'].argmax(1)
    correct += (preds == yb).sum().item()
student_acc = 100 * correct / len(test_ds)
print(f"Student Acc: {student_acc:.2f}%")


Final Test: 100%|██████████| 79/79 [00:08<00:00,  9.27it/s]

Student Acc: 79.92%
